In [40]:
import os
from pathlib import Path
import shutil
from tempfile import mkstemp
from zipfile import ZipFile

from storages.backends.s3boto3 import S3Boto3Storage

from django.core.files.storage import default_storage
from django.contrib.gis.utils import LayerMapping
from django.conf import settings

from public_data.storages import DataStorage
from public_data.models import Ocsge2015, Artificialisee2015to2018

default_storage.listdir("")

(['user_0001', 'user_1'], [])

In [36]:
storage = DataStorage()
storage.listdir(".")

([],
 ['.',
  'A_Brute_2018.zip',
  'Enveloppe_urbaine.zip',
  'OCSGE_2015.zip',
  'OCSGE_2018.zip',
  'Sybarval.zip',
  'Voirire_2018.zip',
  'a_b_2015_2018.zip',
  'a_b_2018_2015.zip',
  'communes_sybarval.zip',
  'emprise_arrondissement_arcachon.zip',
  'zones_baties_2018.zip'])

In [57]:
path = "a_b_2015_2018.zip"
storage.exists(path)

True

In [58]:
temp_dir_path = Path(tempfile.TemporaryDirectory().name)
file_stream = storage.open(path)
with ZipFile(file_stream) as zip_file:
    zip_file.extractall(temp_dir_path)  # extract files to dir

list(temp_dir_path.iterdir())

[PosixPath('/tmp/tmpm5ol4jel/A_B_2015_2018.shp'),
 PosixPath('/tmp/tmpm5ol4jel/A_B_2015_2018.dbf'),
 PosixPath('/tmp/tmpm5ol4jel/A_B_2015_2018.shx'),
 PosixPath('/tmp/tmpm5ol4jel/A_B_2015_2018.prj'),
 PosixPath('/tmp/tmpm5ol4jel/A_B_2015_2018.qpj')]

In [60]:
# get shape file
shp_file = None
for file in temp_dir_path.iterdir():
    if file.is_file() and file.suffix == ".shp":
        shp_file = file
shp_file

PosixPath('/tmp/tmpm5ol4jel/A_B_2015_2018.shp')

In [61]:
cls = Artificialisee2015to2018

cls.objects.all().delete()
lm = LayerMapping(cls, shp_file, cls.mapping)
lm.save(strict=True, verbose=False)

cls.objects.all().count()

3970

In [63]:
# delete temporary folder
if Path.exists(temp_dir_path):
    shutil. rmtree(temp_dir_path)
    
Path.exists(temp_dir_path)

False

In [35]:
# zip all folders within a folder
folders = [p for p in Path("../public_data/media/").iterdir() if p.is_dir()]
for p in folders: 
    shutil.make_archive(f"{p}", "zip", p)